# Compare line segment capacity to PG&E reported No Available Capacity feeders

#### Load the file where no capacity feeders were identified

In [5]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from pyproj import Proj, transform
import contextily as ctx
from shapely.geometry import box
import fiona
import pandas as pd

no_capacity_feeders = pd.read_csv('no_capacity_feeders.csv')

no_capacity_feeders.head()

,feederid,FeederID,geometry
0,83671106,83671106.0,MULTILINESTRING ((587911.275316745 4133626.053...
1,83192102,83192102.0,MULTILINESTRING ((608409.6753358357 4083552.91...
2,82021108,82021108.0,MULTILINESTRING ((590501.5253191572 4120638.53...
3,182331101,182331101.0,MULTILINESTRING ((623731.275350105 4069263.513...
4,42281105,42281105.0,MULTILINESTRING ((490165.74522571266 4350902.7...


#### Load line segment data

In [6]:
line = gpd.read_file("../../data/raw/iou_pge/ICADisplay.gdb", driver='fileGDB', layer="LineDetail")

line.head()

,FeederId,FeederName,Globalid,CSV_LineSection,ICA_Analysis_Date,LoadCapacity_kW,GenCapacity_kW,GenericPVCapacity_kW,GenCapacity_no_OpFlex_kW,GenericCapacity_no_OpFlex_kW,Shape_Length,geometry
0,252081103,TAFT 1103,{3FF9AD05-6886-40FD-BEEC-3BAB9775DCC9},5157787,Jan 2023,0,0,0,0,0,108.633168,"MULTILINESTRING ((825260.426 3895563.863, 8252..."
1,163692101,SALT SPRINGS 2101,{7C49966E-B3CD-4729-A459-F5812F3659BA},4283583,Jun 2021,950,0,0,0,0,6570.616417,"MULTILINESTRING ((747026.635 4261734.433, 7470..."
2,162771101,LAMMERS 1101,{33B4CF19-B9ED-45FC-AB72-D4F32C8D263A},3012323,Apr 2023,0,0,0,1260,1600,15.892972,"MULTILINESTRING ((631046.725 4175942.993, 6310..."
3,012022211,CLAYTON 2211,{4FD0FDBE-C3E4-470C-B9BF-4B85E8709095},5168104,Jun 2023,2940,270,390,4270,5130,205.342666,"MULTILINESTRING ((590111.155 4201648.613, 5901..."
4,163481109,WEBER 1109,{823E2E30-CC87-4410-A14F-561ED1DC75C6},4638156,Aug 2023,100,0,0,1050,1290,73.204776,"MULTILINESTRING ((661267.755 4196894.483, 6612..."


### Lines joined with feeders that have no capacity

In [10]:
# Convert FeederId in the 'line' DataFrame to string
line['FeederId'] = line['FeederId'].astype(str)

# Convert feederid in the 'no_capacity_feeders' DataFrame to string
no_capacity_feeders['feederid'] = no_capacity_feeders['feederid'].astype(str)

# Merge the no_capacity_feeders with the line DataFrame on the 'feederid'
no_capacity_lines = line.drop(columns=['geometry']).merge(
    no_capacity_feeders[['feederid', 'geometry']], 
    left_on='FeederId', # from line
    right_on='feederid', # from no_capacity_feeders
    how='right'
)

# Display the number of merged rows and a preview of the resulting data
print(len(no_capacity_lines['FeederId'].unique()))
print(len(no_capacity_lines))
no_capacity_lines.head()

479
263154


,FeederId,FeederName,Globalid,CSV_LineSection,ICA_Analysis_Date,LoadCapacity_kW,GenCapacity_kW,GenericPVCapacity_kW,GenCapacity_no_OpFlex_kW,GenericCapacity_no_OpFlex_kW,Shape_Length,feederid,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83671106,MULTILINESTRING ((587911.275316745 4133626.053...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83192102,MULTILINESTRING ((608409.6753358357 4083552.91...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82021108,MULTILINESTRING ((590501.5253191572 4120638.53...
3,182331101,GABILAN 1101,{FCE31448-97BF-448C-9E10-C92D75F54149},3056611.0,Feb 2024,0.0,0.0,0.0,0.0,0.0,6.229766,182331101,MULTILINESTRING ((623731.275350105 4069263.513...
4,182331101,GABILAN 1101,{E3608F8C-8DC3-455F-94EC-4231203E4D12},5147422.0,Feb 2024,0.0,0.0,0.0,0.0,0.0,28.991939,182331101,MULTILINESTRING ((623731.275350105 4069263.513...


### There are more than 215 unique feeders which have line segments reporting LoadCapacity_kW as over 1000

In [16]:
# Sort the DataFrame by the 'LoadCapacity_kW' column in descending order
sorted_df = no_capacity_lines.sort_values(by='LoadCapacity_kW', ascending=False)

# Filter the DataFrame to find entries with 'LoadCapacity_kW' greater than 1000.0
above_1000 = sorted_df[sorted_df['LoadCapacity_kW'] > 1000.0]

# Display the number of entries and a preview of the filtered DataFrame
print(f"Number of entries with LoadCapacity_kW > 1000.0: {len(above_1000['FeederId'].unique())}")
above_1000.head()

Number of entries with LoadCapacity_kW > 1000.0: 215


,FeederId,FeederName,Globalid,CSV_LineSection,ICA_Analysis_Date,LoadCapacity_kW,GenCapacity_kW,GenericPVCapacity_kW,GenCapacity_no_OpFlex_kW,GenericCapacity_no_OpFlex_kW,Shape_Length,feederid,geometry
244355,254691102,NORCO 1102,{E43085E7-A887-42A3-BBF6-C8C8EB7EA329},5080485.0,Jan 2024,10010.0,0.0,0.0,11490.0,12000.0,168.963203,254691102,MULTILINESTRING ((835657.4055474764 3908705.03...
67078,153732107,PLUMAS 2107,{374193EC-F747-44B6-B0FD-93CBD5634343},5022560.0,Apr 2023,10000.0,0.0,0.0,7980.0,9910.0,18.236941,153732107,MULTILINESTRING ((625105.6653513853 4318900.26...
67305,153732107,PLUMAS 2107,{C3459DDC-E227-42EF-BF9C-CDE2BC4F1BDC},4279096.0,Apr 2023,10000.0,0.0,0.0,7980.0,9910.0,69.877931,153732107,MULTILINESTRING ((625105.6653513853 4318900.26...
67308,153732107,PLUMAS 2107,{D289F2C3-51C4-4F18-8D7A-814948A08E25},4017853.0,Apr 2023,10000.0,0.0,0.0,7990.0,9920.0,4.570274,153732107,MULTILINESTRING ((625105.6653513853 4318900.26...
67327,153732107,PLUMAS 2107,{8786670A-456B-493E-B2FE-4A2FFDFF7601},3465811.0,Apr 2023,10000.0,0.0,0.0,7980.0,9910.0,38.017760,153732107,MULTILINESTRING ((625105.6653513853 4318900.26...
